# Multiprocessing in Python

What happens if we need to do a lot of computation, and vectorization with NumPy is not enough? 

Basically: How can we assign different tasks to different cores or processors?

## Multiprocessing systems/interfaces

There are a few types of multiprocessing interfaces we can use, in order of easy to difficult:

| Interface | When to use |
|-----------|-------------|
| concurrent.futures.ProcessPoolExecutor | The modern way to launch parallel tasks. Usable for everything that takes a long time and needs to run on only one computer. |
| mpi4py.futures.MPIPoolExecutor | The modern way to run scalable parallel tasks on computer clusters. Use for long tasks that need to run on more than one computer. |
| multiprocess | An improved version of the built-in `multiprocessing` library. A bit more manual than the pools. |
| multiprocessing | The classic built-in library.  You probably will be using a part of this (like the Manager), but other components are trickier than the concurrent.futures implementation. |
| mpi4py classic | A bit beyond the scope of this class. |


In [1]:
import concurrent.futures
import time
import long_functions

In [11]:
def serial_task():
    possible_params = [3, 5, 10, 15]
    starting_time = time.time()
    # for n in possible_params:
    #     long_functions.really_hard_task(n)
    foo = list(map(long_functions.really_hard_task, possible_params))
    duration_time = time.time() - starting_time
    print(f"Took {duration_time:.2f} s")

In [12]:
serial_task()

finished task crunching 3 with result 0.98
finished task crunching 5 with result 0.62
finished task crunching 10 with result 0.69
finished task crunching 15 with result 0.84
Took 8.00 s


In [30]:
def parallel_taskmaster():
    possible_params = [3, 5, 10, 15]
    starting_time = time.time()
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as ppe:
        results = ppe.map(long_functions.really_hard_task, possible_params)
    duration_time = time.time() - starting_time
    print(f"Took {duration_time:.2f} s")
    print(list(results))

In [31]:
parallel_taskmaster()

Took 2.31 s
[{'param': 3, 'duration': 2.000151800000822, 'result': 0.07413610980040808}, {'param': 5, 'duration': 2.0001157999995485, 'result': 0.2157822458749138}, {'param': 10, 'duration': 2.000103999998828, 'result': 0.26036356281686657}, {'param': 15, 'duration': 2.0000798999990366, 'result': 0.370128114336836}]


What happens if we run too many workers at once?

If you want to run a lot of workers, you can try it on DevCloud!

## Basic Linear Algebra Problems in Multiprocessing

Turns out that the Math Kernel Library or any other basic linear algebra system (BLAS) sometimes will try to be sneaky and do multiprocessing/multithreading on its own.  However, this conflicts with your own code, so make sure to do this before importing numpy or any other packages:

```
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

# now you can import numpy and other packages
import numpy as np
import scipy.stats
# etc.
```

## Using multiprocessing for graphical interfaces

Let's try using librosa and PyQtGraph to visualize the short-time Fourier transform (STFT) of a song!